# Модель стабильной динамики

#### Прямая задача:

$$\Psi(f) = \sum_{e \in E \backslash E'} \int\limits_0^{f_e} \tau_e^{\mu}(z) dz + \sum_{e \in E'} f_e \bar t_e \rightarrow \min_{f = \Theta x, x \in X, \\ f_e \leq \bar f_e, ~ e \in E'}$$


#### Двойственная задача:

$$\Upsilon(t) = -  \sum_{w \in W} d_w T_w(t) + <\bar f, t - \bar t> - \mu \sum_{e \in E \backslash E'} h_e^{\mu}(t_e) \rightarrow \min_{t_e \geq \bar t_e, ~ e \in E', \\ t_e \in dom h_e^{\mu}(t_e), ~ e \in E \backslash E'}$$

где $T_w(t)$ - длина кратчайшего пути из $i$ в $j$ ($w = (i,j) \in W$) на графе, ребра которого взвешены вектором $t = \{t_e\}_{e \in E'}$, а функции $h_e^{\mu}(t_e)$ - гладкие вогнутые.

При этом решение изначальной задачи $f$ можно получить из формул:
$$f_e = \bar f_e - s_e, ~ e \in E', ~ \textit{где}~ s_e \geq 0 - \text{множитель Лагранжа к ограничению } t_e \geq \bar t_e $$
$$\tau_e^{\mu}(f_e) = t_e, ~ e \in E \backslash E'$$




### Reading data

In [1]:
import csv
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
net_data = []
with open('Anaheim_net.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t', quotechar='|')
    for ind, row in enumerate(reader):
        if ind > 7:
            net_data = np.append(net_data, np.array(row[1:-1]))
            
net_data = np.array(net_data).reshape((-1,10))
print net_data[:1,:]
num_edges = net_data.shape[0]
print 'Number of edges: ', num_edges

[['1' '117' '9000' '5280' '1.090458488' '0.15' '4' '4842' '0' '1']]
Number of edges:  914


In [3]:
trips_data = []
pairs = []
origin = 0
corrs = []

with open('Anaheim_trips.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for ind, row in enumerate(reader):
        if ind > 3:
            if len(row) > 0 and row[0][0] == 'O':
                origin += 1
            else:
                row = row[:-1]
                if len(row) > 0:
                    row_ = []
                    for elem in row:
                        elem = (str(origin) + ':' + elem).split(':')
                        for ind, sub_elem in enumerate(elem):
                            elem[ind] = sub_elem.strip()
                        row_.append(map(float, elem))
                    trips_data.extend(row_)

print 'Correspondences number: ', len(trips_data), '\n'
print 'Origins number: ', origin
print 'Format: [origin, destination, flow]'
print trips_data[:10]

Correspondences number:  1406 

Origins number:  38
Format: [origin, destination, flow]
[[1.0, 2.0, 1365.9], [1.0, 3.0, 407.4], [1.0, 4.0, 861.4], [1.0, 5.0, 354.4], [1.0, 6.0, 545.1], [1.0, 7.0, 431.5], [1.0, 8.0, 1.0], [1.0, 9.0, 56.8], [1.0, 10.0, 75.3], [1.0, 11.0, 1.0]]


In [4]:
flow_data = []
with open('Anaheim_flow.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t', quotechar='|')
    for ind, row in enumerate(reader):
        if ind > 5:
            for i, elem in enumerate(row):
                row[i] = elem.strip()
            row.remove('')
            row.remove(':')
            row.remove(';')
            #print row
            flow_data = np.append(flow_data, np.array(row))

flow_data = np.array(flow_data).reshape((-1,4))
print flow_data[:5,:]

[['1' '117' '7074.9000000000015' '1.1529198689124767']
 ['2' '87' '9662.5000000000073' '1.3077728285644104']
 ['3' '74' '7668.9999999999927' '1.1766938339006712']
 ['4' '233' '12173.799999999996' '1.6380226412299237']
 ['5' '165' '2586.7999999999956' '1.0915747901668154']]


In [5]:
city = nx.DiGraph()
capacities = np.array(map(float, net_data[:,2]))
weights = np.zeros(num_edges)

for i in xrange(num_edges):
    city.add_edge(int(net_data[i,0]), int(net_data[i,1]), weight=float(net_data[i,3]))
    weights[i] = float(net_data[i,3])
print 'Nodes number: ', len(city.nodes())
print 'Edges number: ', len(city.edges())
#lengths = nx.all_pairs_dijkstra_path_length(city)

Nodes number:  416
Edges number:  914


In [6]:
#trips_data = trips_data[:1000]
print trips_data[:2]

[[1.0, 2.0, 1365.9], [1.0, 3.0, 407.4]]


### Algorithm

In [32]:
import numpy as np
from timeit import default_timer as timer

def weighted_city(t):
    global num_edges, net_data, trips_data
    G = nx.DiGraph()
    for i in xrange(num_edges):
        G.add_edge(int(net_data[i,0]), int(net_data[i,1]), weight=t[i])
    result = 0.
    lengths = nx.all_pairs_dijkstra_path_length(G)
    for i in xrange(len(trips_data)):
        result += lengths[int(trips_data[i][0])][int(trips_data[i][1])] * trips_data[i][2]
    #print 'Weighted winnipeg calculated\n'
    return result

def prox(z):
    return z.dot(z) / 2

def prox_grad(z):
    return z

def V(x, y):
    return prox(x - y)

def f(x):
    global weights, capacities
    return capacities.dot(x - weights) - weighted_city(x)

def grad_d_wT_w(t):
    global num_edges, net_data, trips_data
    G = nx.DiGraph()
    for i in xrange(num_edges):
        G.add_edge(int(net_data[i,0]), int(net_data[i,1]), weight=t[i])
    paths = nx.all_pairs_dijkstra_path(G)
    grad = np.zeros(num_edges)
    for i in xrange(len(trips_data)):
        vec = np.array([ sum([(paths[int(trips_data[i][0])][int(trips_data[i][1])][j] == int(net_data[k,0]) and
                               paths[int(trips_data[i][0])][int(trips_data[i][1])][j + 1] == int(net_data[k,1])) 
                              for j in xrange(len(paths[int(trips_data[i][0])][int(trips_data[i][1])]) - 1)])
                        for k in xrange(num_edges)])
        grad += trips_data[i][2] * vec
    return grad

def grad_f(x):
    global capacities
    return capacities - grad_d_wT_w(x)

def phi(k, x):
    global y, alpha
    res = V(x, y[0])
    for i in xrange(k + 1):
        res += alpha[k] * (f(y[k]) + grad_f(y[k]).dot(x - y[k]))

t0 = timer()
n = num_edges
EPS = 10**-3
MU = 0.25
omega = 1.
MU_tilde = MU / omega

L = [] # list of lists
A = [] # list of numbers
alpha = [] # list of numbers
y = [] # list of arrays

# 0 interation:
L.append([])
L[0].append(2.)
A.append(1. / L[0][0])
alpha.append(1. / L[0][0])


e = np.zeros(n)
e1 = np.ones(n)
y.append(weights)
#!!!!!
grad_f_y_0 = grad_f(y[0])
x_prev = np.maximum(y[0] - alpha[0] * grad_f_y_0, weights)
u = x_prev
#!!!!!
f_y_0 = f(y[0])

j = []
j.append(0)

value = (- f(x_prev) + f_y_0 + grad_f_y_0.dot(x_prev - y[0]) +
         L[0][j[0]] / 2 * np.linalg.norm(x_prev - y[0])**2 + alpha[0] * EPS / (2 * A[0]) )
Y_prev = f(x_prev)

while 0. > value and (j[0] < 10):
    j[0] += 1
    L[0].append(2**(j[0]) * L[0][0])
    A[0] = 1. / L[0][j[0]]
    alpha[0] = 1. / L[0][j[0]]
    x_prev = np.maximum(y[0] - alpha[0] * grad_f_y_0, weights)
    Y_0 = f(x_prev)
    value = (- Y_prev + f_y_0 + grad_f_y_0.dot(x_prev - y[0]) +
         L[0][j[0]] / 2 * np.linalg.norm(x_prev - y[0])**2 + alpha[0] * EPS / (2 * A[0]) )
    print 'j = ', j[0], 'value = ', value,'\n'


np.savetxt('Stable_dynamic_t_data0.txt', x_prev)
print 'Zero iteration is successful! j = ', j[0],' ', Y_prev, '\n'

k = 0
while (k < 100 ):
    j.append(0) #j_k+1
    L.append([])
    L[k + 1].append(L[k][j[k]] / 2) #L_k+1^0
    alpha_next = ( (1. + A[k] * MU_tilde) / (2 * L[k + 1][j[k + 1]]) +
                  np.sqrt( (1. + A[k] * MU_tilde)/(4 * L[k + 1][j[k + 1]]**2) +
                          (A[k] * (1. + A[k] * MU_tilde))/(L[k + 1][j[k + 1]]) ) )
    alpha.append(alpha_next)
    A.append(A[k] + alpha[k + 1])
    y.append((alpha[k + 1] * u + A[k] * x_prev)/A[k+1])
    #!!!!!!!!
    #grad_f_y_next = grad_f(y[k+1])
    #f_y_next = f(y[k + 1])
        
    u_next = np.maximum(u - alpha[k+1] * grad_f(y[k+1]), weights)
    x_next = (alpha[k + 1] * u_next + A[k] * x_prev) / A[k + 1]
    #!!!!!
    #f_x_next = f(x_next)
    
    value = ( f(y[k + 1]) + grad_f(y[k+1]).dot(x_next - y[k + 1]) + 
             (L[k + 1][j[k + 1]] / 2)  * np.linalg.norm(x_next - y[k + 1])**2 +
             alpha[k + 1] * EPS / (2 * A[k + 1])) - f(x_next)
    print value,'\n'
    while (value < 0. and j[k + 1] < 4):
        j[k + 1] += 1
        L[k + 1].append(2**j[k + 1] * L[k][j[k]])
        alpha[k + 1] = ( (1. + A[k] * MU_tilde) / (2 * L[k + 1][j[k + 1]]) +
                        np.sqrt( (1. + A[k] * MU_tilde)/(4 * L[k + 1][j[k + 1]]**2) +
                                (A[k] * (1. + A[k] * MU_tilde))/(L[k + 1][j[k + 1]]) ) )
        A[k+1] = A[k] + alpha[k + 1]
        y[k+1] = (alpha[k + 1] * u + A[k] * x_prev)/A[k+1]
        u_next = np.maximum(u - alpha[k+1] * grad_f(y[k+1]), weights)
        x_next = (alpha[k + 1] * u_next + A[k] * x_prev) / A[k + 1]
        Y_next = f(x_next)
        value = ( f(y[k + 1]) + grad_f(y[k+1]).dot(x_next - y[k + 1]) + 
                 L[k + 1][j[k + 1]] / 2 * np.linalg.norm(x_next - y[k + 1])**2 + 
                 alpha[k + 1] * EPS / (2 * A[k + 1])) - Y_next
        print 'j = ', j[k+1], 'value = ', value
        
    k += 1
    print k, ' iteration done \n'
    DISPERANCY = np.linalg.norm(x_next - x_prev) / np.linalg.norm(x_prev)
    print DISPERANCY
    if DISPERANCY <= 0.01:
        np.savetxt('Stable_dynamic_t_solution' + str(k) + '.txt', x_next)
        print 'Success! Iteration = ', k
        break
    x_prev = x_next
    u = u_next
    Y_prev = Y_next
    np.savetxt('t_data' + str(k) + '.txt', x_next)

t1 = timer()
#print "Solution: ", x_next
print "value: ", Y_next
print 'Time : ', t1 - t0

j =  1 value =  -165561229.007 

j =  2 value =  -104726470.626 

j =  3 value =  -74309091.4351 

j =  4 value =  -59100401.8398 

j =  5 value =  -51496057.0421 

j =  6 value =  -47693884.6433 

j =  7 value =  -45792798.4439 

j =  8 value =  -44842255.3442 

j =  9 value =  -44366983.7943 

j =  10 value =  -44129348.0194 

Zero iteration is successful! j =  10   -4467820902.96 

448967.201924 

1  iteration done 

0.117801832365
914903.267091 

2  iteration done 

0.0262842429642
2280966.02644 

3  iteration done 

0.00847774178244
Success! Iteration =  3
value:  -4891628267.71
Time :  201.857620114


# Модель Бэкмана

**BPR-функция (Bureau of Public Road)**
$$ t_e(f_e) = t_e^0 \left(1 + \gamma \left(\frac{f_e}{\bar f_e} \right)^{\frac1{\mu}} \right),$$

Для BPR-функции $\sigma^*(t_e) = \bar f_e \left(\frac{t_e - \bar t_e}{\bar t_e  \gamma} \right)^{\mu} \left( \frac{t_e - \bar t_e}{1 + \mu}\right)$.

#### Прямая задача:

$$\Psi(f) = \sum_{e \in E} \sigma_e(f_e) = \sum_{e \in E} \int\limits_0^{f_e} \tau_e(z) dz \rightarrow \min_{f = \Theta x, x \in X}$$

#### Двойственная задача:

$$\Upsilon(t) = -\sum_{w \in W} d_w T_w(t) + \sum_{e \in E} \bar f_e \left(\frac{t_e - \bar t_e}{\bar t_e  \gamma} \right)^{\mu} \left( \frac{t_e - \bar t_e}{1 + \mu}\right)  \rightarrow \min_{t_e \in dom~\sigma_e(t_e),~e \in E}$$

### Algorithm

In [30]:
import numpy as np
from timeit import default_timer as timer

def weighted_city(t):
    global num_edges, net_data, trips_data
    G = nx.DiGraph()
    for i in xrange(num_edges):
        G.add_edge(int(net_data[i,0]), int(net_data[i,1]), weight=t[i])
    result = 0.
    lengths = nx.all_pairs_dijkstra_path_length(G)
    for i in xrange(len(trips_data)):
        result += lengths[int(trips_data[i][0])][int(trips_data[i][1])] * trips_data[i][2]
    #print 'Weighted winnipeg calculated\n'
    return result

def prox(z):
    return z.dot(z) / 2

def prox_grad(z):
    return z

def V(x, y):
    return prox(x - y)

def f(x):
    global MU, gamma, weights, capacities
    return sum( np.array([capacities[i] * ((x[i] - weights[i]) / (gamma * weights[i]))**MU * (x[i] - weights[i]) / (1 + MU) 
              for i in xrange(x.shape[0])]) )

def grad_d_wT_w(t):
    global num_edges, net_data, trips_data
    G = nx.DiGraph()
    for i in xrange(num_edges):
        G.add_edge(int(net_data[i,0]), int(net_data[i,1]), weight=t[i])
    paths = nx.all_pairs_dijkstra_path(G)
    grad = np.zeros(num_edges)
    for i in xrange(len(trips_data)):
        vec = np.array([ sum([(paths[int(trips_data[i][0])][int(trips_data[i][1])][j] == int(net_data[k,0]) and
                               paths[int(trips_data[i][0])][int(trips_data[i][1])][j + 1] == int(net_data[k,1])) 
                              for j in xrange(len(paths[int(trips_data[i][0])][int(trips_data[i][1])]) - 1)])
                        for k in xrange(num_edges)])
        grad += trips_data[i][2] * vec
    return grad

def grad_f(x):
    global MU, gamma
    grad_sigma = np.array([capacities[i] * ( 1./(1 + MU) * ((x[i] - weights[i]) / (gamma * weights[i]))**MU 
                                                          + (x[i] - weights[i]) / (1. + MU) * MU / (weights[i] * gamma) 
                                                          * ((x[i] - weights[i]) / (gamma * weights[i]))**(MU - 1.))
            for i in xrange(x.shape[0])])
    return - grad_d_wT_w(x) + grad_sigma

def phi(k, x):
    global y, alpha
    res = V(x, y[0])
    for i in xrange(k + 1):
        res += alpha[k] * (f(y[k]) + grad_f(y[k]).dot(x - y[k]))

t0 = timer()
n = num_edges
EPS = 10**-3
MU = 0.25
omega = 1.
MU_tilde = MU / omega
gamma = 0.5

L = [] # list of lists
A = [] # list of numbers
alpha = [] # list of numbers
y = [] # list of arrays

# 0 interation:
L.append([])
L[0].append(2.)
A.append(1. / L[0][0])
alpha.append(1. / L[0][0])


e = np.zeros(n)
e1 = np.ones(n)
y.append(weights * 1.01)
#!!!!!
grad_f_y_0 = grad_f(y[0])
x_prev = np.maximum(y[0] - alpha[0] * grad_f_y_0, weights)
u = x_prev
#!!!!!
f_y_0 = f(y[0])

j = []
j.append(0)

value = (- f(x_prev) + f_y_0 + grad_f_y_0.dot(x_prev - y[0]) +
         L[0][j[0]] / 2 * np.linalg.norm(x_prev - y[0])**2 + alpha[0] * EPS / (2 * A[0]) )
Y_prev = f(x_prev)

while 0. > value and (j[0] < 10):
    j[0] += 1
    L[0].append(2**(j[0]) * L[0][0])
    A[0] = 1. / L[0][j[0]]
    alpha[0] = 1. / L[0][j[0]]
    x_prev = np.maximum(y[0] - alpha[0] * grad_f_y_0, weights)
    Y_0 = f(x_prev)
    value = (- Y_prev + f_y_0 + grad_f_y_0.dot(x_prev - y[0]) +
         L[0][j[0]] / 2 * np.linalg.norm(x_prev - y[0])**2 + alpha[0] * EPS / (2 * A[0]) )
    print 'j = ', j[0], 'value = ', value,'\n'


np.savetxt('Bacman_t_data0.txt', x_prev)
print 'Zero iteration is successful! j = ', j[0],' ', Y_prev, '\n'

k = 0
while (k < 100 ):
    j.append(0) #j_k+1
    L.append([])
    L[k + 1].append(L[k][j[k]] / 2) #L_k+1^0
    alpha_next = ( (1. + A[k] * MU_tilde) / (2 * L[k + 1][j[k + 1]]) +
                  np.sqrt( (1. + A[k] * MU_tilde)/(4 * L[k + 1][j[k + 1]]**2) +
                          (A[k] * (1. + A[k] * MU_tilde))/(L[k + 1][j[k + 1]]) ) )
    alpha.append(alpha_next)
    A.append(A[k] + alpha[k + 1])
    y.append((alpha[k + 1] * u + A[k] * x_prev)/A[k+1])
    #!!!!!!!!
    #grad_f_y_next = grad_f(y[k+1])
    #f_y_next = f(y[k + 1])
        
    u_next = np.maximum(u - alpha[k+1] * grad_f(y[k+1]), weights)
    x_next = (alpha[k + 1] * u_next + A[k] * x_prev) / A[k + 1]
    #!!!!!
    #f_x_next = f(x_next)
    Y_next = f(x_next)
    value = ( f(y[k + 1]) + grad_f(y[k+1]).dot(x_next - y[k + 1]) + 
             (L[k + 1][j[k + 1]] / 2)  * np.linalg.norm(x_next - y[k + 1])**2 +
             alpha[k + 1] * EPS / (2 * A[k + 1])) - Y_next
    print value,'\n'
    while (value < 0. and j[k + 1] < 4):
        j[k + 1] += 1
        L[k + 1].append(2**j[k + 1] * L[k][j[k]])
        alpha[k + 1] = ( (1. + A[k] * MU_tilde) / (2 * L[k + 1][j[k + 1]]) +
                        np.sqrt( (1. + A[k] * MU_tilde)/(4 * L[k + 1][j[k + 1]]**2) +
                                (A[k] * (1. + A[k] * MU_tilde))/(L[k + 1][j[k + 1]]) ) )
        A[k+1] = A[k] + alpha[k + 1]
        y[k+1] = (alpha[k + 1] * u + A[k] * x_prev)/A[k+1]
        u_next = np.maximum(u - alpha[k+1] * grad_f(y[k+1]), weights)
        x_next = (alpha[k + 1] * u_next + A[k] * x_prev) / A[k + 1]
        Y_next = f(x_next)
        value = ( f(y[k + 1]) + grad_f(y[k+1]).dot(x_next - y[k + 1]) + 
                 L[k + 1][j[k + 1]] / 2 * np.linalg.norm(x_next - y[k + 1])**2 + 
                 alpha[k + 1] * EPS / (2 * A[k + 1])) - Y_next
        print 'j = ', j[k+1], 'value = ', value
        
    #if k == 0:
    #    INITIAL_DICPERANCY = np.abs(Y_next - Y_prev) / np.abs(Y_prev)
    k += 1
    print k, ' iteration done \n'
    #DISPERANCY = np.abs(Y_next - Y_prev) / np.abs(Y_prev)
    DISPERANCY = np.linalg.norm(x_next - x_prev) / np.linalg.norm(x_prev)
    print DISPERANCY
    if DISPERANCY <= 0.01:
        np.savetxt('Bacman_t_solution' + str(k) + '.txt', x_next)
        print 'Success! Iteration = ', k
        break
    x_prev = x_next
    Y_prev = Y_next
    u = u_next
    np.savetxt('Bacman_t_data' + str(k) + '.txt', x_next)

t1 = timer()
#print "Solution: ", x_next
print "value: ", Y_next
print 'Time : ', t1 - t0

j =  1 value =  -3738912063.2 

j =  2 value =  -3444464626.11 

j =  3 value =  -3295195268.41 

j =  4 value =  -3216882009.48 

j =  5 value =  -3173699938.8 

j =  6 value =  -3149743282.39 

j =  7 value =  -3136739047.38 

j =  8 value =  -3130012224.91 

j =  9 value =  -3126642420.53 

j =  10 value =  -3124955303.48 

Zero iteration is successful! j =  10   3167814089.17 

4397552.89142 

1  iteration done 

0.255574377584
10016041.1862 

2  iteration done 

0.0521182464367
21323335.9197 

3  iteration done 

0.014787277548
44355460.0054 

4  iteration done 

0.00677112843804
Success! Iteration =  4
value:  2887572141.81
Time :  262.718924493
